<a href="https://colab.research.google.com/github/quanvu0996/compet/blob/master/Kaggle_compet/WiDS_2021/wids_quanvh8_run_for_stacking_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ENVIROMENT SETTING

In [1]:
# COLAB ONLY
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install octopus-ml
!pip install catboost
!pip install bayesian-optimization
!pip install missingpy 

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import octopus_ml as oc
import sklearn as sk

from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn import preprocessing
from sklearn import ensemble, naive_bayes, svm, linear_model
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score,roc_curve, auc, confusion_matrix
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import PCA, TruncatedSVD

from multiprocessing.pool import ThreadPool
from multiprocessing import Pool 
from scipy.stats import ks_2samp

from xgboost import XGBClassifier
from catboost import Pool, cv, CatBoostClassifier, CatBoostRegressor
from lightgbm import LGBMClassifier
import lightgbm as lgb

# from pytorch_tabnet.tab_model import TabNetClassifier
import torch

import glob

from bayes_opt import BayesianOptimization
import missingpy 
import warnings
warnings.filterwarnings("ignore")

# COMMON FUNCTION

In [4]:
# Function lưu kết quả thành 3 file cho stacking
# submit thành 3 file: train, valid, test prediction; y_pred của valid và y_true_valid sẽ được dùng để train 
def _to_save_model_predict_3files(model, file_name):
    X_train['prediction'] = model.predict(X_train)
    X_train['diabetes_mellitus'] = y_train

    X_valid['prediction'] = model.predict(X_valid)
    X_valid['diabetes_mellitus'] = y_valid

    df_test['diabetes_mellitus'] = model.predict( df_test[cols_ft] )

    # save to csv
    X_train[['prediction', 'diabetes_mellitus']].to_csv(folder_path +'/WiDS2021/ensemble_outputs/stacking_train/train_'+file_name+'.csv', index = True)
    X_valid[['prediction', 'diabetes_mellitus']].to_csv(folder_path +'/WiDS2021/ensemble_outputs/stacking_valid/valid_'+file_name+'.csv', index = True)
    df_test[ 'diabetes_mellitus'].to_csv(folder_path +'/WiDS2021/ensemble_outputs/stacking_test/test_'+file_name+'.csv', index = True)
    print('results saved')

In [5]:
# Convert data 64bit to 32bit if possible to reduce memory
def reduce_mem_usage(df: pd.DataFrame,
                     verbose: bool = True) -> pd.DataFrame:
    
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2

    for col in df.columns:
        col_type = df[col].dtypes

        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()

            if str(col_type)[:3] == 'int':

                if (c_min > np.iinfo(np.int32).min
                      and c_max < np.iinfo(np.int32).max):
                    df[col] = df[col].astype(np.int32)
                elif (c_min > np.iinfo(np.int64).min
                      and c_max < np.iinfo(np.int64).max):
                    df[col] = df[col].astype(np.int64)
            else:
                if (c_min > np.finfo(np.float16).min
                        and c_max < np.finfo(np.float16).max):
                    df[col] = df[col].astype(np.float16)
                elif (c_min > np.finfo(np.float32).min
                      and c_max < np.finfo(np.float32).max):
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)

    end_mem = df.memory_usage().sum() / 1024**2
    reduction = (start_mem - end_mem) / start_mem

    msg = f'Mem. usage decreased to {end_mem:5.2f} MB ({reduction * 100:.1f} % reduction)'
    if verbose:
        print(msg)

    return df

# BỘ DATA V5 VỚI MISSING VALUES ĐOÁN BẰNG MODEL

In [6]:
folder_path = '/content/drive/MyDrive/Data/colabs_data/WiDS'
df_full = reduce_mem_usage(
            pd.read_csv(folder_path + "/WiDS2021/Wids2021_full_feature_v5_updated_20210228.csv") )
df_full.set_index('encounter_id', inplace = True)

Mem. usage decreased to 479.58 MB (66.5 % reduction)


In [17]:
# Convert boolean columns
cols_boolean = df_full.select_dtypes('bool').columns
df_full[cols_boolean] = df_full[cols_boolean].astype('int8')
df_full.replace([np.inf, -np.inf], 0, inplace = True)

In [26]:
# Data type 
col_target = 'diabetes_mellitus'
cols_id = ['hospital_id','encounter_id', 'Unnamed: 0', 'Unnamed: 0.1','knn_value_16', 'knn_value_512', 'knn_value_64' ]# , 'total_chronic', 'has_complicator', 'has_history', 'apache_hist'
cols_cat = [i for i in df_full.select_dtypes('object').columns if i not in cols_id + [col_target]]
cols_num = [i for i in df_full if i not in cols_cat + cols_id + [col_target] ]
cols_ft = cols_cat + cols_num

In [21]:
# Split data set
df_train, df_test = df_full[:130157], df_full[130157:]
X_train, X_valid, y_train, y_valid = train_test_split( 
    df_train[cols_ft], df_train[col_target], train_size= 98000, random_state= 68)

### Check dữ liệu bộ v5 with missing filled

In [9]:
df_full['apache_2_diagnosis'].unique()

array([113. , 108. , 122. , 203. , 119. , 301. , 116. , 112. , 303. ,
       218. , 304. , 302. , 305. , 124. , 202. , 207. , 110. , 209. ,
       109. , 106. , 117. , 120. , 193.9, 217. , 114. , 102. , 308. ,
       105. , 212. , 219. , 306. , 121. , 214. , 123. , 213. , 208. ,
       101. , 118. , 307. , 215. , 192.6, 192.4, 103. , 192.2, 115. ,
       194.8, 104. , 216. , 107. , 193.6, 192.8, 192.5, 193.5, 194.9,
       193.8, 190. , 189.9], dtype=float16)

In [7]:
df_full.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 140391 entries, 0 to 140390
Columns: 1394 entries, Unnamed: 0 to knn_value_512
dtypes: bool(64), float16(979), float32(35), float64(32), int32(277), int64(1), object(6)
memory usage: 479.6+ MB


In [16]:
df_full['encounter_id']

0         214826
1         246060
2         276985
3         262220
4         201746
           ...  
140386    143750
140387    143813
140388    137126
140389    135652
140390    136852
Name: encounter_id, Length: 140391, dtype: int32

In [25]:
# x = df_full.isnull().sum()
x[x>0]

gender                                                 71
hospital_admit_source                               35931
diasbp_noninvasive_tot_change_value_range_normed     1729
mbp_invasive_tot_change_value_range_normed            696
mbp_tot_change_value_range_normed                    2018
mbp_noninvasive_tot_change_value_range_normed        1762
calcium_tot_change_value_range_normed                4908
wbc_tot_change_value_range_normed                    4710
arterial_po2_tot_change_value_range_normed           6453
platelets_tot_change_value_range_normed              4493
spo2_tot_change_value_range_normed                   4211
heartrate_tot_change_value_range_normed              1291
diasbp_tot_change_value_range_normed                 1870
glucose_tot_change_value_range_normed                2832
sysbp_invasive_tot_change_value_range_normed          415
potassium_tot_change_value_range_normed              4365
lactate_tot_change_value_range_normed                5412
sodium_tot_cha

# BỘ DATA 2020FEATURES KAGGLE

In [5]:
# Bộ feature 2020
folder_path = '/content/drive/MyDrive/Data/colabs_data/WiDS'
# derived features
df_train = pd.read_csv(folder_path + "/WiDS2021/train_df_2020features.csv")
df_test = pd.read_csv(folder_path + "/WiDS2021/test_df_2020features.csv")

# Đổi dạng bool thành int
cols_boolean = df_train.select_dtypes(include= 'boolean').columns
df_train[cols_boolean] = df_train[cols_boolean].astype('int')
df_test[cols_boolean] = df_test[cols_boolean].astype('int')
df_train.replace([np.inf, -np.inf], 0, inplace = True)
df_test.replace([np.inf, -np.inf], 0, inplace = True)

# Data type 
col_target = 'diabetes_mellitus'
cols_id = ['encounter_id', 'Unnamed: 0', 'Unnamed: 0.1','knn_value_16', 'knn_value_512', 'knn_value_64' ]
cols_cat = [i for i in df_train.select_dtypes('object').columns if i not in cols_id + [col_target]]
cols_num = [i for i in df_train if i not in cols_cat + cols_id + [col_target] ]
cols_ft = cols_cat + cols_num

df_train.set_index('encounter_id', inplace = True)
df_test.set_index('encounter_id', inplace = True)

In [6]:
X_train, X_valid, y_train, y_valid = train_test_split( 
    df_train[cols_ft], df_train[col_target], train_size= 98000, random_state= 68)

In [13]:
X_train

,intubated_apache,glucose_wb,d1_arterial_po2_max,gcs_function,cat_bmi,d1_diasbp_invasive_max,h1_arterial_po2_min,h1_bilirubin_min,gcs_motor_apache,hepatic_failure,h1_creatinine_max,h1_calcium_max,h1_diasbp_invasive_min,h1_sodium_min,d1_arterial_po2_min,h1_hco3_min,d1_arterial_pco2_max,gcs_eyes_apache,hospital_admit_source,h1_bun_min,d1_bilirubin_min,icu_admit_source,d1_sysbp_noninvasive_min,h1_potassium_max,h1_sysbp_invasive_min,h1_hemaglobin_max,h1_arterial_ph_max,icu_stay_type,d1_lactate_min,h1_calcium_min,bilirubin_apache,cirrhosis,h1_diasbp_invasive_max,solid_tumor_with_metastasis,gender,d1_spo2_min,h1_potassium_min,gcs_verbal_apache,d1_diasbp_invasive_min,comorbidity_score,...,glucose_indicator,hematocrit_indicator,pao2fio2ratio_indicator,arterial_ph_indicator,resprate_indicator,sodium_indicator,temp_indicator,wbc_indicator,creatinine_indicator,total_day_more_extreme,d1_resprate_div_mbp_min,d1_resprate_div_sysbp_min,d1_lactate_min_div_diasbp_min,d1_heartrate_min_div_d1_sysbp_min,d1_hco3_div,d1_resprate_times_resprate,left_average_spo2,total_chronic,total_cancer_immuno,has_complicator,apache_3j,rank_frqenc_elective_surgery,rank_frqenc_icu_id,rank_frqenc_arf_apache,rank_frqenc_intubated_apache,rank_frqenc_ventilated_apache,rank_frqenc_cirrhosis,rank_frqenc_hepatic_failure,rank_frqenc_immunosuppression,rank_frqenc_solid_tumor_with_metastasis,rank_frqenc_apache_3j_diagnosis_x,rank_frqenc_apache_2_diagnosis_x,rank_frqenc_apache_3j,rank_frqenc_apache_3j_diagnosis_split1,rank_frqenc_apache_2_diagnosis_split1,rank_frqenc_gcs_sum_type,rank_frqenc_total_cancer_immuno,rank_frqenc_total_chronic,rank_frqenc_apache_2_diagnosis_type,rank_frqenc_apache_3j_diagnosis_type
encounter_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
249554,1,4,316.0,12,4,0.0,0.0,0.0,4,0,0.0,0.0,0.0,0.0,69.0,0.0,49.0,2,0,0.0,0.3,0,78.0,0.0,0.0,0.0,0.00,0,0.5,0.0,0.3,0,0.0,0,0,73.0,0.0,0,0.0,0,...,0,0,0,0,0,0,0,0,0,10,0.3777,0.21790,0.01428,0.6284,1.167,510.0,91.0,0,0,0,5,83574.5,41171.5,72116.5,11061.0,23186.0,71304.0,71132.5,71982.0,71643.5,57477.5,36594.0,3594.5,49912.5,7818.0,36690.5,73512.5,71757.5,83594.0,10676.5
156140,0,3,0.0,4,2,0.0,0.0,0.0,5,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3,0,0.0,0.6,0,94.0,0.0,0.0,0.0,0.00,0,0.0,0.0,0.6,0,0.0,0,1,90.0,0.0,3,0.0,0,...,1,1,0,0,0,1,0,1,1,10,0.3215,0.19150,0.00000,0.9683,1.000,648.0,96.0,0,0,0,6,83574.5,8846.5,72116.5,81256.5,93381.5,71304.0,71132.5,71982.0,71643.5,72153.5,69659.5,67044.0,18443.5,124466.5,91465.0,73512.5,71757.5,83594.0,84044.0
243930,0,3,0.0,5,4,74.0,0.0,0.0,5,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3,4,0.0,1.0,0,102.0,0.0,0.0,0.0,0.00,0,0.0,0.0,1.0,0,0.0,0,0,99.0,0.0,4,74.0,0,...,1,1,0,0,0,1,0,1,1,10,0.2222,0.15690,0.00000,0.4510,1.000,400.0,99.7,0,0,0,0,83574.5,27224.5,72116.5,81256.5,93381.5,71304.0,71132.5,71982.0,71643.5,123720.5,99329.5,124531.5,98782.5,94324.5,91465.0,73512.5,71757.5,120512.0,124972.5
216008,0,0,0.0,10,6,0.0,0.0,0.0,3,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0,0.0,0.0,0,106.0,0.0,0.0,0.0,0.00,0,0.0,0.0,0.0,0,0.0,0,1,93.0,0.0,0,0.0,0,...,0,0,0,0,0,0,0,0,0,7,0.2222,0.16980,0.00000,0.7734,0.000,360.0,97.7,0,0,0,3,83574.5,81598.5,72116.5,81256.5,93381.5,71304.0,71132.5,71982.0,71643.5,50996.5,117582.0,12565.5,98782.5,55646.0,9663.0,73512.5,71757.5,83594.0,68275.5
269221,0,3,84.0,0,5,0.0,84.0,0.0,5,0,0.0,0.0,0.0,0.0,84.0,0.0,59.7,2,4,0.0,0.0,2,85.0,0.0,0.0,11.5,7.26,0,0.0,0.0,0.0,0,0.0,0,1,92.0,0.0,0,0.0,0,...,0,0,0,0,0,0,1,1,0,12,0.1380,0.09410,0.00000,1.0700,1.131,192.0,97.3,0,0,0,6,83574.5,128109.5,72116.5,81256.5,23186.0,71304.0,71132.5,71982.0,71643.5,59988.5,27492.5,67044.0,98782.5,94324.5,36690.5,73512.5,71757.5,13105.5,84044.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
194102,1,4,12

# ERROR FIXING

In [9]:
file_name = 'lgbm_dart_ft2020'
ss_train = pd.read_csv(folder_path +'/WiDS2021/ensemble_outputs/stacking_train/train_'+file_name+'.csv')
ss_valid = pd.read_csv(folder_path +'/WiDS2021/ensemble_outputs/stacking_valid/valid_'+file_name+'.csv')
ss_test = pd.read_csv(folder_path +'/WiDS2021/ensemble_outputs/stacking_test/test_'+file_name+'.csv')

In [20]:
ss_valid.index = X_valid.index
ss_test.index = df_test.index

In [22]:

# save to csv
ss_train[['prediction', 'diabetes_mellitus']].to_csv(folder_path +'/WiDS2021/ensemble_outputs/stacking_train/train_'+file_name+'.csv', index = True)
ss_valid[['prediction', 'diabetes_mellitus']].to_csv(folder_path +'/WiDS2021/ensemble_outputs/stacking_valid/valid_'+file_name+'.csv', index = True)
ss_test[ 'diabetes_mellitus'].to_csv(folder_path +'/WiDS2021/ensemble_outputs/stacking_test/test_'+file_name+'.csv', index = True)